## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import math
import time

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
from mPE_fn import mPE
from scipy.spatial import distance
from scipy.stats import entropy

## Load Data

In [2]:
# loading entire high-dimensional trajectory
path = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/normal/traj_1/trajectories/'
trajectories = os.listdir(path)

## Data Cleansing

In [3]:
# removing invalid values (e.g. NaN)
# input data is already normalized (z-score) but needs to get rid of non-valued datapoints
nan_cols = []
for i, time_bin in enumerate(trajectories):
    trajectory = loadmat(path + time_bin)
    trajectory = trajectory['trajectory'] 
    for i in range(trajectory.shape[1]):
        if np.isnan(trajectory[:, i]).all():
            nan_cols.append(i)

sampling_factor = 30
nan_cols = np.asarray(nan_cols)
if len(np.where(nan_cols==nan_cols[0])[0])*3 == len(nan_cols):
    sampled_trajectories = np.zeros(((trajectory.shape[0]*int(len(trajectories)/sampling_factor)), trajectory.shape[1]-3))
    all_trajectories = np.zeros((trajectory.shape[0]*int(len(trajectories)), trajectory.shape[1]-3))
    a = 0
    for i, time_bin in enumerate(trajectories):
        trajectory = loadmat(path + time_bin)
        trajectory = trajectory['trajectory'] 
        trajectory = np.delete(trajectory, nan_cols, 1)
        idx = a*trajectory.shape[0]
        idx_2 = i*trajectory.shape[0]
        all_trajectories[idx_2:idx_2+trajectory.shape[0], 0:sampled_trajectories.shape[1]] = trajectory
        if i % sampling_factor == 0 and sampled_trajectories.shape[0]-idx >= trajectory.shape[0]:
            sampled_trajectories[idx:idx+trajectory.shape[0], 0:sampled_trajectories.shape[1]] = trajectory
            a+=1
        
print(sampled_trajectories.shape)
print(all_trajectories.shape)

(216000, 57)
(8100000, 57)


## First and Second Moment Analysis

In [ ]:
# mean 
means = [np.mean(all_trajectories[:, i]) for i in range(all_trajectories.shape[1])]

# standard deviation
stds = [np.std(all_trajectories[:, i]) for i in range(all_trajectories.shape[1])]
max_dim_std = np.where(np.isclose(stds, max(stds)))


## Subsampling Frequency (using MI)

### Helper Functions

In [4]:
def probability(sequence, decimals=0):
    '''
    input: 
        - 1D sequence of rv observations
    return: 
        - probability vector
    '''
    
    if sequence.shape[0] < sequence.shape[1]:
        sequence = np.transpose(sequence)
    
    # round input sequence to avoid sparse probability vector
    sequence = np.round(sequence, decimals)
    unique = np.unique(sequence, axis=0)
    n_triplets = len(unique)

    # fill probability vector
    prob_vector = np.zeros((n_triplets, 1))
    for row in sequence:
        occurrences = len(np.where(np.all(np.isclose(sequence, row), axis=1))[0])
        idx = np.where(np.all(np.isclose(unique, row), axis=1))
        if prob_vector[idx[0]] == 0:
            prob_vector[idx[0]] = occurrences/(sequence.shape[0])
    return prob_vector

In [5]:
def joint_probability(*argv):
    '''
    input: 
        - sequence_1 of rv_1 observations
        - sequence_2 of rv_2 observations
        - number of dimensions of input sequences to consider(default = all)
    return: 
        - joint probability vector
    '''
    n_args = len(argv)
    if n_args == 2:
        s1 = argv[0]
        s2 = argv[1]
        dims = s1.shape[1]
        decimals = 0
        
    if n_args == 3:
        s1 = argv[0]
        s2 = argv[1]
        dims = argv[2]
        # select dims based on input
        s1 = s1[:, 0:dims]
        s2 = s2[:, 0:dims]
        decimals = 0
        
    # checking that the dimensions of the input sequences are in the right order
    if s1.shape[0] < s1.shape[1]:
        s1 = np.transpose(s1)
    if s2.shape[0] < s2.shape[1]:
        s2 = np.transpose(s2)
    
    s1 = np.around(s1, decimals)
    s2 = np.around(s2, decimals) 
    
    # here we assume that the input sequences are already rounded (n_observations x n_dimensions)
    unique_s1 = np.unique(s1, axis=0)
    n_triplets_s1 = len(unique_s1)
    unique_s2 = np.unique(s2, axis=0)
    n_triplets_s2 = len(unique_s2)
    
    joint_data = np.concatenate((s1, s2), axis=1)
    print('JOINT SPACE SIZE: ', joint_data.shape)
    
    # filling joint probability matrix
    joint_prob_matrix = np.zeros((n_triplets_s1, n_triplets_s2))
    occurrences, idx_s1, idx_s2 = 0, 0, 0
    for joint_array in joint_data:
        occurrences = len(np.where(np.all(np.isclose(joint_data, joint_array), axis=1))[0])
        idx_s1 = np.where(np.all(np.isclose(unique_s1, joint_array[0:dims]), axis=1))
        idx_s2 = np.where(np.all(np.isclose(unique_s2, joint_array[dims:2*dims]), axis=1))
        if joint_prob_matrix[idx_s1[0][0], idx_s2[0][0]] == 0:
            joint_prob_matrix[idx_s1[0][0], idx_s2[0][0]] = (occurrences/joint_data.shape[0])
            
    return joint_prob_matrix

In [6]:
def conditional_entropy(prob_s1s2, prob_s2):
    E_cond = 0
    for i in range(prob_s1s2.shape[0]):
        for j in range(prob_s1s2.shape[1]):
            E_cond += prob_s1s2[i,j] * math.log((prob_s2[j]/prob_s1s2[i,j]), 2)
            
    return E_cond

In [7]:
def joint_entropy(prob_s1s2):
    E_joint = 0
    for i in range(prob_s1s2.shape[0]):
        for j in range(prob_s1s2.shape[1]):
            if prob_s1s2[i,j] > 0:
                E_joint += prob_s1s2[i,j] * math.log((1/prob_s1s2[i,j]), 2)
            
    return E_joint

### Sub-sampling frequency vs MI

In [ ]:
fs_og = 300
length_ = int(all_trajectories.shape[0]/10)
sub_fs = np.arange(1,50, 5)
names = ['HeadF', 'HeadB', 'HeadL', 'SpineF', 'SpineM',
         'SpineL', 'Offset1', 'Offset2', 'HipL', 'HipR',
         'ElbowL', 'ArmL', 'ShoulderL', 'ShoulderR', 'ElbowR'
         'ArmR', 'KneeR', 'KneeL', 'ShinL', 'ShinR']
markers = [11, 15]
MIs = np.zeros((len(sub_fs), 1))
for i, fs in enumerate(sub_fs):
    # step for sampling frequency
    step = int(300/fs);
    
    # construct sample trajectories for the first 2 dims ('HeadF' and 'HeadB')
    s1 = all_trajectories[0:length_:step, dims[0]*3:(dims[0]*3)+3]
    s2 = all_trajectories[0:length_:step, dims[1]*3:(dims[1]*3)+3]
            
    # probability vectors for the two sequences and joint probability
    print('CALCULATING PROBABILITY VECTORS...')
    prob_s1 = probability(s1)
    prob_s2 = probability(s2)
    print('CALCULATING JOINT PROBABILITY VECTORS...') 
    prob_s1s2 = joint_probability(s1, s2) 
    print(np.sum(prob_s1s2))
    
    # calculating entropy values for MI
    print('CALCULATING ENTROPY VECTORS...')
    E_s1 = entropy(prob_s1, base=2)
    E_s2 = entropy(prob_s2, base=2)
    E_s1_cond_s2 = joint_entropy(prob_s1s2) - E_s2
    print(E_s1, E_s2, E_s1_cond_s2)
    MI = E_s1 - E_s1_cond_s2
    MIs[i] = MI
    
# plotting relation between MI and sub-sampling frequency
plt.scatter(sub_fs, MIs)
plt.ylabel('MI')
plt.xlabel('Sub-sampling frequency')
plt.show()

CALCULATING PROBABILITY VECTORS...
CALCULATING JOINT PROBABILITY VECTORS...
JOINT SPACE SIZE:  (2700, 6)
0.9999999999999999
CALCULATING ENTROPY VECTORS...
[4.88101572] [4.82060177] [1.58048517]
CALCULATING PROBABILITY VECTORS...
CALCULATING JOINT PROBABILITY VECTORS...
JOINT SPACE SIZE:  (16200, 6)
1.0
CALCULATING ENTROPY VECTORS...
[4.89540654] [4.85592927] [1.68080895]
CALCULATING PROBABILITY VECTORS...
CALCULATING JOINT PROBABILITY VECTORS...
JOINT SPACE SIZE:  (30000, 6)
1.0
CALCULATING ENTROPY VECTORS...
[4.90227779] [4.8573105] [1.70022187]
CALCULATING PROBABILITY VECTORS...
CALCULATING JOINT PROBABILITY VECTORS...
JOINT SPACE SIZE:  (45000, 6)
0.9999999999999999
CALCULATING ENTROPY VECTORS...
[4.90326487] [4.85655709] [1.70678899]
CALCULATING PROBABILITY VECTORS...
CALCULATING JOINT PROBABILITY VECTORS...
JOINT SPACE SIZE:  (57858, 6)
0.9999999999999999
CALCULATING ENTROPY VECTORS...
[4.90194096] [4.85717843] [1.70501746]
CALCULATING PROBABILITY VECTORS...
CALCULATING JOINT PROB

## Dimensionality Reduction: PCA

In [ ]:
# inspecting inter-dimensional variance with PCA
pca = PCA()
pca.fit(all_trajectories)

plt.scatter(np.arange(all_trajectories.shape[1]), pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_[0:3]))
plt.ylabel('Expalined Variance Ratio')
plt.xlabel('Dimension')
plt.show()

# reduce data according to explained variance values using linear PCA
pca = PCA(n_components=3)
reduced_traj = pca.fit_transform(all_trajectories)
reduced_traj.shape

print(np.cov(np.transpose(reduced_traj)))

## Dimensionality Reduction: MDS

In [ ]:
embedding = MDS(n_components=3)
reduced_traj_MDS = embedding.fit_transform(sampled_trajectories[0:3600, :])
reduced_traj_MDS.shape

print(np.cov(np.transpose(reduced_traj_MDS)))

## Dimensionality Reduction: Kernel-PCA

In [ ]:
kPCA = KernelPCA(n_components=3, kernel = 'linear')
reduced_traj_k = kPCA.fit_transform((all_trajectories))
reduced_traj_k.shape

## Incremental Covariance Matrix

In [ ]:
cov_init = np.cov(np.transpose(first_trajectories))
cov_sampled = np.cov(np.transpose(sampled_trajectories))
vars_ = np.zeros((cov_init.shape[0], 2))
means_ = np.zeros((first_trajectories.shape[1],1))
for i in range(cov_init.shape[0]):
    vars_[i, 0] = cov_init[i,i]
    vars_[i, 1] = cov_sampled[i,i]
    means_[i] = np.mean(first_trajectories[:, i])

# update initial covariance
n_init = first_trajectories.shape[0]
for i, time_bin in enumerate(trajectories):
    if i >= sampling_factor:
        print('trajectory: ', i)
        print(path + time_bin)
        trajectory = loadmat(path + time_bin)
        trajectory = trajectory['trajectory'] 
        trajectory = np.delete(trajectory, nan_cols, 1)
        for a, row in enumerate(trajectory):
            for j, obs in enumerate(row):
                new_mean_j = (n_init*means_[j] + obs)/(n_init + 1)
                for k in range(j, len(row)):
                    cov_temp = cov_init[j,k]
                    new_mean_k = (n_init*means_[k] + row[k])/(n_init + 1)
                    cov_init[j,k] = (n_init/n_init + 1)*cov_temp + ((1/n_init)*(obs-new_mean_j)*(row[k]-new_mean_k))
                    cov_init[k,j] = cov_init[j,k]
                                                                    
            n_init = n_init+1

## Data Exploration

### Helper Function 

In [ ]:
def mov_average(data, window_length, shift_length):
    av_data = []
    a = 0
    for i in range(0,len(data),shift_length):
        if i>=window_length:
            av_data.append(np.mean(data[i-window_length:i]))
            a = a + 1
    return av_data

In [ ]:
sample_x_traj = all_trajectories[:, 0]
sample_x_traj = mov_average(sample_x_traj, 9000, 1)

plt.plot(np.arange(len(sample_x_traj)), sample_x_traj)

## Entropy Analysis

### Helper functions

In [6]:
def prob_vector(raw_data, max_size, min_, decimals=1):
    rounded_data = [(round(x * 10**(decimals)))/(10**decimals) for x in raw_data]
    sorted_data = np.sort(rounded_data)
    sorted_data = np.array(sorted_data, dtype=np.float64)

    already_seen = []
    prob_vector = np.zeros((max_size+1))
    occurrences = 0
    idx = 0
    for datapoint in sorted_data:
        if datapoint not in already_seen:
            occurrences = np.count_nonzero(sorted_data == datapoint)
            already_seen.append(datapoint)
            idx = int((datapoint - min_)/(1/10**decimals))
            prob_vector[idx] = (occurrences/len(sorted_data))
    
    return prob_vector

In [7]:
def get_mPE_matrix(reduced_traj, bins_number, traj_number, orders=[3]):
    mPE_vector = np.zeros((bins_number, traj_number, len(orders)))
    traj_length = int((reduced_traj.shape[0]/bins_number)/traj_number)
    for a, order in enumerate(orders):
        for i in range(bins_number):
            idx_1 = 0
            idx = 0
            for j in range(0, traj_length*traj_number, traj_length):
                idx_1 = i*traj_number*traj_length 
                traj = reduced_traj[idx_1 + j: idx_1 + j + traj_length]
                if traj.shape[0]>0:
                    [HH, _]=mPE(traj, order)
                    mPE_vector[i, idx, a] = HH
                idx +=1
    return mPE_vector

In [8]:
def get_vel_matrix(trajectory, bins_number, traj_number, least_varaince_zdim=8*3):
    f_s = 300
    vel_matrix = np.zeros((bins_number, traj_number))
    traj_length = int((reduced_traj.shape[0]/bins_number)/traj_number)
    for i in range(bins_number):
        idx = 0
        idx_1 = 0
        for j in range(0, traj_length*traj_number, traj_length):
            idx_1 = i*traj_number*traj_length 
            traj = trajectory[idx_1 + j: idx_1 + j + traj_length, least_varaince_zdim:least_varaince_zdim+3]
            vel_bin = 0
            last_point = traj[0, :]
            for point in traj:
                vel_bin = vel_bin + distance.euclidean(point, last_point)
                last_point = point
            vel_matrix[i, idx] = vel_bin
            idx += 1
                
    return vel_matrix

In [9]:
def behavioural_entropy(behav_time_series, bins_number, traj_number, mode):
    mPE_vector = np.zeros((bins_number, traj_number, len(orders)))
    traj_length = int((reduced_traj.shape[0]/bins_number)/traj_number)
    for a, order in enumerate(orders):
        for i in range(bins_number):
            idx = 0
            idx_1 = 0
            for j in range(0, traj_length*traj_number, traj_length):
                idx_1 = i*bins_number*traj_length 
                traj = reduced_traj[idx_1 + j: idx_1 + j + traj_length]
                if mode=='PE':
                    [HH,hh_norm]=mPE(traj, order)
                elif mode=='Shannon':
                    HH = entropy(traj, base=2)
                mPE_vector[i, idx , a] = HH
                idx += 1
                
    return mPE_vector

In [10]:
def joint_prob_matrix(x, y, n_behaviours, truncation_decimal=1):
    # Checking that the dimensions are in the right order
    if x.shape[0] < x.shape[1]:
        x = np.transpose(x)
    if y.shape[0] < y.shape[1]:
        y = np.transpose(y)
        
    # reducing position array to 2D [PC1, PC2] to avoid spoarse probability matrix
    x = x[:, 0:2]
    # rounding reduced trajectory to 2 decimal place
    rounded_x = [(round(i * 10**(decimals)))/(10**decimals) for i in x]
    rounded_x = np.asarray(rounded_x)
    # finding max and min of reduced trajectory data
    min_ = float('inf')
    max_ = float('-inf')
    for time_point in range(rounded_x.shape[0]):
        for positions in range(rounded_x.shape[1]):
            if rounded_x[time_point, positions] > max_:
                max_ = rounded_x[time_point, positions]
            elif rounded_x[time_point, positions] < min_:
                min_ = rounded_x[time_point, positions]

    min_ = (round(min_ * 10**(truncation_decimal)))/(10**truncation_decimal)
    max_ = (round(max_ * 10**(truncation_decimal)))/(10**truncation_decimal)
    # finding the maximimum number of elements in the rounded position matrix
    max_size = int((max_ - min_)/(1/(10**truncation_decimal)))
    # concatating joint data (x and y) --> (each row of joint data consits of [PC1, PC2, behaviour index])
    joint_data = np.concatenate(rounded_x, y, axis=1)
    
    # filling joint probability matrix
    already_seen = []
    joint_prob_matrix = 2*np.ones((max_size, n_behaviours))
    occurrences, idx_location, idx_behaviour = 0, 0, 0
    for joint_array in rounded_x:
        if joint_array not in already_seen:
            occurrences = np.count_nonzero(sorted_data == joint_array)
            already_seen.append(joint_array)
            found = True
            while found == False:
                idx_location = np.random.randint(max_size)
                if joint_prob_matrix[idx_location, idx_behaviour] == 2:
                    found = False
                    break
            idx_behaviour = joint_array[joint_data.shape[1]]
            joint_prob_matrix[idx_location, idx_behaviour] = (occurrences/joint_data.shape[0])
    
    return joint_prob_matrix

### Entropy time-evolution

In [11]:
# Entropy Calculation:
# - Divide entire trajetcory in 'bins_number' bins
# - Divide each bin for 'traj_number' trajectories
# - Calculate mPE for each of these trajectories 
minutes = 6
f_s = 300
bin_length = f_s*60*minutes
bins_number = int(reduced_traj.shape[0]/bin_length)
traj_number = 30
orders = [3, 5]

print(reduced_traj.shape[0], bins_number, traj_number)

mPE_vector = get_mPE_matrix(reduced_traj, bins_number, traj_number, orders)

8100000 75 30


In [ ]:
# entropy distribution visualization

# plotting mpE of the first bin 
mode = 'distribution'
bin_n = 2
order_n = 0
fig = plt.figure()


# plotting histograms of two different bins
if mode == 'hist':
    n, bins, patches = plt.hist(x = mPE_vector[bin_n, :, order_n] , bins='auto', color='green',
                                alpha=0.7, rwidth=0.85)
    n, bins, patches = plt.hist(x= mPE_vector[mPE_vector.shape[0]-5, :, order_n] , bins='auto', color='red',
                                alpha=0.7, rwidth=0.85)

# plotting mPE continuous distribution as a function of time 
elif mode == 'distribution':
    means = np.zeros(((bins_number),1))
    stds = np.zeros(((bins_number),1))
    for bin_n in range(bins_number):
        means[bin_n] = np.mean(mPE_vector[bin_n, :, order_n])
        stds[bin_n] = np.std(mPE_vector[bin_n, :, order_n])
        # ax = sns.displot(mPE_vector[bin_n, :, order_n],  kind="kde")        
        ax = sns.distplot(mPE_vector[bin_n, :, order_n], hist=False, kde=True, 
                          bins=0.1, 
                          hist_kws={'edgecolor':'black'},
                          kde_kws={'linewidth': 4})
        
plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.xlabel('mPE')
plt.ylabel('Probability')

fig = plt.figure()
plt.plot(np.arange(bins_number), means)
plt.xlabel('time bin')
plt.ylabel('mean mPE')

fig = plt.figure()
plt.plot(np.arange(bins_number), stds)
plt.xlabel('time bin')
plt.ylabel('mean mPE')

In [ ]:
min_ = float('inf')
max_ = float('-inf')
for order in range(mPE_vector.shape[2]):
    for bin_ in range(mPE_vector.shape[0]):
        for traj in range(mPE_vector.shape[1]):
            if mPE_vector[bin_, traj, order] > max_:
                max_ = mPE_vector[bin_, traj, order]
            elif mPE_vector[bin_, traj, order] < min_:
                min_ = mPE_vector[bin_, traj, order]

truncation_decimal = 1
min_ = (round(min_ * 10**(truncation_decimal)))/(10**truncation_decimal)
max_ = (round(max_ * 10**(truncation_decimal)))/(10**truncation_decimal)
max_size = int((max_ - min_)/(1/(10**truncation_decimal)))

# plotting J-S divergence as a function of bin number
compare_to = prob_vector(mPE_vector[0, :, 0], max_size, min_)
js_vector = np.zeros((mPE_vector.shape[0]))
ks_vector = np.zeros((mPE_vector.shape[0], 2))
ks_and_div = np.zeros((mPE_vector.shape[0], 2))
significance_lev = 0.01
for i in range(mPE_vector.shape[0]):
    js_vector[i] = distance.jensenshannon(prob_vector(mPE_vector[i, :, 0], max_size, min_), compare_to)
    ks_and_div[i, 0] = js_vector[i]
    if i > 0:
        ks_vector[i, :] = stats.ks_2samp(mPE_vector[i, :, 0], mPE_vector[0, :, 0])
        if ks_vector[i, 1] <= significance_lev:
            ks_and_div[i, 1] = 1
        else:
            ks_and_div[i, 1] = 0
            
fig = plt.figure()
# plotting velocity profile of mouse (point-wise)
plt.plot(np.arange(mPE_vector.shape[0]), js_vector)
plt.xlabel('bin number')
plt.ylabel('J-S Divergence')

js_rejected = np.where(ks_and_div[:,1] == 1)
js_th = float('inf')
for i in js_rejected[0]:
    if ks_and_div[i, 0] < js_th:
        js_th = ks_and_div[i, 0]
    
fig = plt.figure()
plt.scatter(ks_and_div[:, 0], ks_and_div[:, 1])
plt.axhline(y=significance_lev, xmin=0, xmax=1)
plt.axvline(x=js_th, ymin=0, ymax=1.2)
plt.xlabel('J-S Divergence')
plt.ylabel('P-Value')

### Velocity vs Entropy

In [ ]:
# Calculate instantaneous velocity of each datapoint (maybe only consider x-y dims)
lest_varaince_dim = 8*3
minutes = 5
fs = 300
bin_length = fs*60*minutes

bins_number = int(reduced_traj.shape[0]/bin_length)
traj_number = 30
orders = [5]
vel_matrix = get_vel_matrix(all_trajectories, bins_number, traj_number)

In [ ]:
# plotting bin-velocity against corresponding mPE
print(bins_number)
print(mPE_vector.shape)
vel_vector = vel_matrix.flatten()
mPE_vector_ = mPE_vector.flatten()
p = np.polyfit(mPE_vector_, vel_vector, 2)
x_new = np.linspace(4,12,200)
ffit = np.polyval(p, x_new)

fig = plt.figure()
plt.scatter(mPE_vector_, vel_vector)
plt.xlabel('mPE')
plt.ylabel('velocity')
plt.plot(x_new, ffit)